## Whisperを使用したインタラクティブPhi 3 Mini 4K Instructチャットボット

### イントロダクション:
インタラクティブPhi 3 Mini 4K Instructチャットボットは、ユーザーがテキストまたは音声入力を使用してMicrosoft Phi 3 Mini 4K Instructデモと対話できるツールです。このチャットボットは、翻訳、天気の更新、一般的な情報収集など、さまざまなタスクに使用できます。

In [ ]:
# 必要なPythonパッケージをインストール
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Cudaサポートが利用可能かどうかを確認
# 出力がTrueの場合 = Cuda
# 出力がFalseの場合 = Cudaなし（モデルをGPUで実行するにはCudaのインストールが必要）
import os 
import torch
print(torch.cuda.is_available())


[Huggingfaceアクセス トークンを作成](https://huggingface.co/settings/tokens)

新しいトークンを作成 
新しい名前を提供 
書き込み権限を選択
トークンをコピーして安全な場所に保存


次のPythonコードは、2つの主要なタスクを実行します：`os`モジュールのインポートと環境変数の設定。

1. `os`モジュールのインポート：
   - Pythonの`os`モジュールは、オペレーティングシステムと対話する方法を提供します。環境変数へのアクセス、ファイルやディレクトリの操作など、さまざまなオペレーティングシステム関連のタスクを実行できます。
   - このコードでは、`import`文を使用して`os`モジュールをインポートしています。この文により、`os`モジュールの機能が現在のPythonスクリプトで使用可能になります。

2. 環境変数の設定：
   - 環境変数は、オペレーティングシステム上で実行されるプログラムがアクセスできる値です。これは、複数のプログラムで使用できる設定やその他の情報を保存する方法です。
   - このコードでは、`os.environ`ディクショナリを使用して新しい環境変数を設定しています。ディクショナリのキーは`'HF_TOKEN'`で、値は`HUGGINGFACE_TOKEN`変数から取得されます。
   - `HUGGINGFACE_TOKEN`変数は、このコードスニペットの上に定義されており、`#@param`構文を使用して文字列値`"hf_**************"`に割り当てられています。この構文は、Jupyterノートブックでよく使用され、ノートブックインターフェイスでユーザー入力とパラメータ設定を直接行うことができます。
   - `'HF_TOKEN'`環境変数を設定することで、プログラムの他の部分や同じオペレーティングシステム上で実行される他のプログラムがアクセスできるようになります。

全体として、このコードは`os`モジュールをインポートし、`HUGGINGFACE_TOKEN`変数に提供された値を使用して`'HF_TOKEN'`という名前の環境変数を設定します。

In [ ]:
import os
# Hugging Faceトークンを設定
# Hugging Faceトークンを環境変数に追加
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN'] = HUGGINGFACE_TOKEN

In [ ]:
# Phi-3-mini-4k-instructモデルとWhisper Tinyをダウンロード
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# 音声からテキストへの変換用のwhisper
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Edge TTSサービスを使用してテキストを音声に変換（TTS）します。以下の関数実装を一つずつ見ていきましょう：

1. `calculate_rate_string(input_value)`：この関数は入力値を受け取り、TTS音声の速度文字列を計算します。入力値は音声の速度を表し、値1は通常の速度を表します。関数は入力値から1を引き、それを100倍し、入力値が1以上かどうかに基づいて符号を決定します。関数は速度文字列を"{sign}{rate}"の形式で返します。

2. `make_chunks(input_text, language)`：この関数は入力テキストと言語をパラメータとして受け取ります。言語固有のルールに基づいて入力テキストをチャンクに分割します。この実装では、言語が"English"の場合、関数は各ピリオド（"."）でテキストを分割し、先頭または末尾の空白を削除します。次に、各チャンクにピリオドを追加し、フィルタリングされたチャンクのリストを返します。

3. `tts_file_name(text)`：この関数は入力テキストに基づいてTTS音声ファイルのファイル名を生成します。テキストに対していくつかの変換を行います：末尾のピリオドを削除（存在する場合）、テキストを小文字に変換、先頭と末尾の空白を削除し、スペースをアンダースコアに置き換えます。次に、テキストを最大25文字に切り詰め（長い場合）、または空の場合は完全なテキストを使用します。最後に、[`uuid`]モジュールを使用してランダムな文字列を生成し、それを切り詰めたテキストと組み合わせてファイル名を作成します。形式は"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"です。

4. `merge_audio_files(audio_paths, output_path)`：この関数は複数の音声ファイルを1つの音声ファイルに結合します。音声ファイルのパスのリストと出力パスをパラメータとして受け取ります。関数は空の`AudioSegment`オブジェクトを初期化し、[`merged_audio`]と呼びます。次に、各音声ファイルのパスを反復処理し、`pydub`ライブラリの`AudioSegment.from_file()`メソッドを使用して音声ファイルをロードし、現在の音声ファイルを[`merged_audio`]オブジェクトに追加します。最後に、結合された音声を指定された出力パスにMP3形式でエクスポートします。

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`：この関数はEdge TTSサービスを使用してTTS操作を実行します。テキストチャンクのリスト、音声の速度、音声名、および保存パスをパラメータとして受け取ります。チャンクの数が1より大きい場合、関数は個々のチャンクの音声ファイルを保存するためのディレクトリを作成します。次に、各チャンクを反復処理し、`calculate_rate_string()`関数、音声名、およびチャンクテキストを使用してEdge TTSコマンドを構築し、`os.system()`関数を使用してコマンドを実行します。コマンドの実行が成功した場合、生成された音声ファイルのパスをリストに追加します。すべてのチャンクを処理した後、`merge_audio_files()`関数を使用して個々の音声ファイルを結合し、結合された音声を指定された保存パスに保存します。チャンクが1つだけの場合、Edge TTSコマンドを直接生成し、音声を保存パスに保存します。最後に、生成された音声ファイルの保存パスを返します。

6. `random_audio_name_generate()`：この関数は[`uuid`]モジュールを使用してランダムな音声ファイル名を生成します。ランダムなUUIDを生成し、それを文字列に変換し、最初の8文字を取り、".mp3"拡張子を追加してランダムな音声ファイル名を返します。

7. `talk(input_text)`：この関数はTTS操作を実行するための主要なエントリポイントです。入力テキストをパラメータとして受け取ります。まず、入力テキストの長さを確認して、それが長い文（600文字以上）かどうかを判断します。長さと`translate_text_flag`変数の値に基づいて、言語を決定し、`make_chunks()`関数を使用してテキストチャンクのリストを生成します。次に、`random_audio_name_generate()`関数を使用して音声ファイルの保存パスを生成します。最後に、`edge_free_tts()`関数を呼び出してTTS操作を実行し、生成された音声ファイルの保存パスを返します。

全体として、これらの関数は協力して入力テキストをチャンクに分割し、音声ファイルのファイル名を生成し、Edge TTSサービスを使用してTTS操作を実行し、個々の音声ファイルを1つの音声ファイルに結合します。

In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # 空のAudioSegmentを初期化
    merged_audio = AudioSegment.silent(duration=0)

    # 各音声ファイルのパスを反復処理
    for audio_path in audio_paths:
        # Pydubを使用して音声ファイルをロード
        audio = AudioSegment.from_file(audio_path)

        # 現在の音声ファイルをmerged_audioに追加
        merged_audio += audio

    # 結合された音声を指定された出力パスにエクスポート
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# 音声の性別を男性から女性に変更し、使用したい音声を選択
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

2つの関数`convert_to_text`と`run_text_prompt`の実装、および2つのクラス`str`と`Audio`の宣言。

関数`convert_to_text`は、`audio_path`を入力として受け取り、`whisper_model`というモデルを使用して音声をテキストに変換します。関数はまず、`gpu`フラグが`True`に設定されているかどうかを確認します。もしそうであれば、`whisper_model`を使用し、`word_timestamps=True`、`fp16=True`、`language='English'`、`task='translate'`などのパラメータを設定します。`gpu`フラグが`False`の場合、`fp16=False`を使用して`whisper_model`を使用します。生成された転写結果は`scan.txt`というファイルに保存され、テキストとして返されます。

関数`run_text_prompt`は、メッセージと`chat_history`を入力として受け取ります。`phi_demo`関数を使用して、入力メッセージに基づいてチャットボットからの応答を生成します。生成された応答は`talk`関数に渡され、応答が音声ファイルに変換され、ファイルパスが返されます。`Audio`クラスは音声ファイルを表示および再生するために使用されます。音声は`IPython.display`モジュールの`display`関数を使用して表示され、`autoplay=True`パラメータを使用して`Audio`オブジェクトが作成されるため、音声は自動的に再生されます。`chat_history`は入力メッセージと生成された応答で更新され、空の文字列と更新された`chat_history`が返されます。

`str`クラスは、文字のシーケンスを表すPythonの組み込みクラスです。文字列の操作や処理のためのさまざまなメソッドを提供します。これらのメソッドには、`capitalize`、`casefold`、`center`、`count`、`encode`、`endswith`、`expandtabs`、`find`、`format`、`index`、`isalnum`、`isalpha`、`isascii`、`isdecimal`、`isdigit`、`isidentifier`、`islower`、`isnumeric`、`isprintable`、`isspace`、`istitle`、`isupper`、`join`、`ljust`、`lower`、`lstrip`、`partition`、`replace`、`removeprefix`、`removesuffix`、`rfind`、`rindex`、`rjust`、`rpartition`、`rsplit`、`rstrip`、`split`、`splitlines`、`startswith`、`strip`、`swapcase`、`title`、`translate`、`upper`、`zfill`などがあります。これらのメソッドを使用して、文字列の検索、置換、フォーマット、および操作を行うことができます。

`Audio`クラスは、音声オブジェクトを表すカスタムクラスです。Jupyter Notebook環境で音声プレーヤーを作成するために使用されます。このクラスは、`data`、`filename`、`url`、`embed`、`rate`、`autoplay`、および`normalize`などのさまざまなパラメータを受け入れます。`data`パラメータは、numpy配列、サンプルのリスト、ファイル名またはURLを表す文字列、または生のPCMデータである可能性があります。`filename`パラメータは、音声データをロードするローカルファイルを指定するために使用され、`url`パラメータは、音声データをダウンロードするURLを指定するために使用されます。`embed`パラメータは、音声データをデータURIを使用して埋め込むか、元のソースから参照するかを決定します。`rate`パラメータは、音声データのサンプリングレートを指定します。`autoplay`パラメータは、音声が自動的に再生を開始するかどうかを決定します。`normalize`パラメータは、音声データが最大可能範囲に正規化（再スケーリング）されるかどうかを指定します。`Audio`クラスは、ファイルまたはURLから音声データを再ロードするための`reload`などのメソッドや、HTMLの音声要素の対応する属性を取得するための`src_attr`、`autoplay_attr`、および`element_id_attr`などの属性も提供します。

全体として、これらの関数とクラスは、音声をテキストに転写し、チャットボットからの音声応答を生成し、Jupyter Notebook環境で音声を表示および再生するために使用されます。

In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)